In [1]:
import pympi
import os
from collections import defaultdict, Counter
from tqdm.auto import tqdm
import pandas as pd

In [2]:
eaf_files = [file for file in os.listdir('./Разметки eaf/переразмеченное') if file.endswith('eaf')]

In [3]:
clfs = defaultdict(list)
for eaf in tqdm(eaf_files):
    eafob = pympi.Elan.Eaf('./Разметки eaf/переразмеченное/' + eaf)
    tiers = [tier for tier in eafob.get_tier_names() if 'глосс_clf' in tier]
    for tier in tiers:
        for sign in eafob.get_annotation_data_for_tier(tier):
            noun_shape_verb = sign[2].split(':')
            clfs[eaf].append((sign[0], sign[1], tuple(noun_shape_verb[1:])))

  0%|          | 0/5 [00:00<?, ?it/s]

In [13]:
clfs_all = Counter()
for file, clf in clfs.items():
    for c in clf:
        clfs_all[c[-1]] += 1

In [14]:
# было уникальных 242
print("Уникальных:", len(clfs_all))

Уникальных: 162


In [6]:
clfs_all

Counter({('бальзам', '137', 'сглаживать'): 3,
         ('волосы', '87', 'торчать'): 1,
         ('волосы', '86', 'торчать'): 1,
         ('вода', '87', 'стекать'): 1,
         ('волосы', '95', 'трогать'): 2,
         ('голова', '93', 'быть'): 1,
         ('волосы', '73', 'быть'): 1,
         ('голова', '73', 'быть'): 1,
         ('волосы', '87', 'быть'): 1,
         ('бальзам', '136', 'держать'): 1,
         ('пар', '93', 'обжигать'): 1,
         ('люди', '87', 'продвигаться'): 2,
         ('объект', '71', 'взлетать'): 2,
         ('самолет', '68', 'взлетать'): 1,
         ('самолет', '68', 'приземляться'): 1,
         ('выключатель', '24', 'включить'): 1,
         ('выключатель', '24', 'выключить'): 1,
         ('куртка', '1', 'надеть'): 2,
         ('люди', '87', 'двигаться'): 2,
         ('чемодан', '30', 'схватить'): 1,
         ('сумка', '30', 'бежать.с'): 2,
         ('поезд', '69', 'ехать'): 1,
         ('человек', '59', 'вскочить'): 1,
         ('человек', '3', 'идти'): 2,
    

In [7]:
shapes = [s for n, s, v in clfs_all.keys()]

In [8]:
df_shapes = pd.DataFrame()
df_shapes.index = sorted(set(shapes))

In [9]:
shapes_nouns = defaultdict(set)
shapes_verbs = defaultdict(set)
shapes_n_v = defaultdict(set)
for n, s, v in clfs_all.keys():
    shapes_nouns[s].add(n)
    shapes_verbs[s].add(v)
    shapes_n_v[s].add(n+':'+v)

for d in [shapes_nouns, shapes_verbs, shapes_n_v]:
    for shape, l in d.items():
        d[shape] = ', '.join(l)

In [10]:
shapes_nouns, shapes_verbs, shapes_n_v

(defaultdict(set,
             {'137': 'бальзам',
              '87': 'холм, медвежонок, вода, люди, дерево, гора, волосы',
              '86': 'волосы',
              '95': 'беремог, инородный.предмет, зубы, желудок, еда, мышцы, волосы, бактерии',
              '93': 'мыщца, здание, пар, голова, желчный.пузырь, воздух, пузырь',
              '73': 'холм, голова, желудок, спинка.стула, волосы, берег',
              '136': 'сэндвич, бальзам, бочка',
              '71': 'лес, дельфин, земля, человек, берег, тело, льдина, преграда, язык, объект, желудок, орел, рука, вода, виноград, контейнер, сэндвич, книга, лапа, спина, бочка, поезд, дверь',
              '68': 'самолет',
              '24': 'таз, выключатель, ботинок',
              '1': 'голова, дверца.духовки, рука, кулак, объект, перчатка, куртка, человек, горизонтальная поверхность',
              '30': 'сумка, лейка, чемодан',
              '69': 'снег, поезд',
              '59': 'медведь, медвежонок, беремот, бегемот, чудовище, ч

In [11]:
for d, name in zip([shapes_nouns, shapes_verbs, shapes_n_v],
                  ['nouns', 'verbs', 'n_v']):
    df_ = pd.DataFrame.from_dict(d, orient='index', columns=[name])
    df_shapes = pd.concat([df_shapes, df_], axis=1)
    df_shapes[f'n_{name}'] = df_shapes[name].apply(lambda x: len(x.split(', ')))
df_shapes.head()

,nouns,n_nouns,verbs,n_verbs,n_v,n_n_v
1,"голова, дверца.духовки, рука, кулак, объект, п...",9,"надеть, поднимать, наклонить, нажимать.наверх,...",12,"голова:ударить, объект:отпускать, перчатка:над...",15
13,"орел, медвежонок, еда, дельфин",4,"летать, плыть, пропустить.в, идти, быть",5,"дельфин:плыть, дельфин:быть, медвежонок:идти, ...",5
136,"сэндвич, бальзам, бочка",3,"держать, быть, плыть, откусить",4,"бочка:плыть, сэндвич:откусить, бочка:быть, бал...",4
137,бальзам,1,сглаживать,1,бальзам:сглаживать,1
138,"льдина, кусок.льдины",2,"отколоться, плыть, быть",3,"льдина:плыть, кусок.льдины:отколоться, льдина:...",3


In [12]:
df_shapes.to_csv('clfs_reannotated_summary.csv')